In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt

In [ ]:
!wget http://andrewjanowczyk.com/wp-static/lymphoma.tar.gz
!tar -zxvf lymphoma.tar.gz
!mkdir lymphoma
!mv CLL lymphoma
!mv FL lymphoma
!mv MCL lymphoma
os.listdir('.')
!rm -r lymphoma/FL/.DS_Store

In [4]:
path=os.path.join("/kaggle/input","lymphoma")
listdi =os.listdir(path)
for i in listdi:
    print(i)

FL
MCL
CLL


In [ ]:
patch_size=224
patches_array=[]
print(listdi)
count=-1;

labels=[]
for i in listdi:
    count=count+1
    print(os.path.join("/kaggle/input/lymphoma",i))
    str_di="/kaggle/working/"+str(i)
    print(str_di)
    img_count=0
    for j in os.listdir(os.path.join("/kaggle/input/lymphoma",i)):
        img_array = cv2.imread(os.path.join(os.path.join("/kaggle/input/lymphoma",i),j),cv2.IMREAD_UNCHANGED)
        new_array = np.array(img_array)
        row=0        
        for m in range(int(len(img_array)/patch_size)):
            col=0
            for n in range(int(len(img_array[0])/patch_size)):
                img_count=img_count+1
                patch=new_array[row:row+patch_size,col:col+patch_size]
                col=col+patch_size
                patches_array.append(patch)
                cv2.imwrite(os.path.join(str_di,str(img_count))+".png",patch)
                labels.append(count)
            row=row+patch_size         
plt.imshow(patches_array[10])
plt.show()
labels=tf.keras.utils.to_categorical(labels,num_classes=3)

['FL', 'MCL', 'CLL']
/kaggle/input/lymphoma/FL
/kaggle/working/FL
/kaggle/input/lymphoma/MCL
/kaggle/working/MCL
/kaggle/input/lymphoma/CLL
/kaggle/working/CLL


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

data_gen = ImageDataGenerator(validation_split=0.25,rotation_range=20)
path = "/kaggle/working"
train_data_generator  = data_gen.flow_from_directory(path,target_size=(patch_size,patch_size),classes=["FL","CLL","MCL"],class_mode='categorical',color_mode='rgb',subset='training')
validation_data_generator = data_gen.flow_from_directory(path,target_size=(patch_size,patch_size),classes=["FL","CLL","MCL"],class_mode='categorical',color_mode='rgb',subset='validation')


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import add

model = Sequential()
model.add(Conv2D(32, (5, 5), strides = 1, padding = "same", activation = "relu", input_shape=(patch_size,patch_size,3)))
model.add(Dropout(0.4))
model.add(Conv2D(32, (5, 5), strides = 1, padding = "same", activation = "relu"))
model.add(Dropout(0.4))
model.add(MaxPooling2D((3, 3)))

model.add(Conv2D(64, (5, 5), strides = 1, padding = "same", activation = "relu"))
model.add(Dropout(0.4))
model.add(Conv2D(64, (5, 5), strides = 1, padding = "same", activation = "relu"))
model.add(Dropout(0.4))
model.add(MaxPooling2D((3, 3)))

model.add(Conv2D(128, (5, 5), strides = 1, padding = "same", activation = "relu"))
model.add(Dropout(0.4))
model.add(Conv2D(128, (5, 5), strides = 1, padding = "same", activation = "relu"))
model.add(Dropout(0.4))
model.add(MaxPooling2D((3, 3)))

model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64,activation='relu'))
model.add(Dense(3, activation = "softmax"))

model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
model.summary()



In [ ]:
import keras
mod = model.fit_generator(train_data_generator,epochs=10,validation_data=validation_data_generator,shuffle=True,callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3),keras.callbacks.callbacks.ModelCheckpoint('/kaggle/working/Parameters',monitor='val_loss',save_best_only=True,save_weights_only=True)])

In [2]:
for i in ['FL','MCL','CLL']:
    os.mkdir(os.path.join("/kaggle/working",i))



In [3]:
os.mkdir(os.path.join("/kaggle/working","Parameters"))